# 딥러닝을 위한 고급 도구

## Sequential 모델을 넘어서: 케라스의 함수형 API
* Sequential 모델은 네트워크 입력과 출력이 하나라고 가정한다.
* 최근에 개발된 많은 신경망 구조는 선형적이지 않은 네크워크 토폴로지(topology)가 필요하다.
* 예를 들어 (구글의 세게디 등이 개발한) **인셉션** 모듈을 사용하는 인셉션 계열의 네트워크들이다.
* 모델에 **잔차 연결**을 추가하는 경향도 있다.
* 케라스에는 훨씬 더 일반적이고 유연한 다른 방법인 **함수형 API**가 있다.

### 함수형 API 소개
* **함수형 API**(functional API)에서는 직접 텐서들의 입출력을 다룬다.

In [1]:
from keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

In [4]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 32)                1056      
                                                                 
 dense_12 (Dense)            (None, 10)                330       
                                                                 
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 2s 3ms/step - loss: 12.7100
Epoch 2/10
8/8 [==============================] - 0s 5ms/step - loss: 13.2682
Epoch 3/10
8/8 [==============================] - 0s 3ms/step - loss: 14.6229
Epoch 4/10
8/8 [==============================] - 0s 3ms/step - loss: 16.4729
Epoch 5/10
8/8 [==============================] - 0s 4ms/step - loss: 18.8948
Epoch 6/10
8/8 [==============================] - 0s 3ms/step - loss: 21.6221
Epoch 7/10
8/8 [==============================] - 0s 3ms/step - loss: 24.7408
Epoch 8/10
8/8 [==============================] - 0s 4ms/step - loss: 27.9899
Epoch 9/10
8/8 [==============================] - 0s 3ms/step - loss: 31.6988
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 37.8111


In [6]:
score

37.81106948852539

### 다중 입력 모델

In [7]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['acc']
)

In [8]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)

answers = to_categorical(answers)

model.fit([text, question], answers, epochs=10, batch_size=128)

model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 3s 27ms/step - loss: 6.2147 - acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 0s 13ms/step - loss: 6.1970 - acc: 0.0450
Epoch 3/10
8/8 [==============================] - 0s 13ms/step - loss: 6.1442 - acc: 0.0090
Epoch 4/10
8/8 [==============================] - 0s 12ms/step - loss: 6.0507 - acc: 0.0050
Epoch 5/10
8/8 [==============================] - 0s 12ms/step - loss: 5.9817 - acc: 0.0080
Epoch 6/10
8/8 [==============================] - 0s 12ms/step - loss: 5.8877 - acc: 0.0110
Epoch 7/10
8/8 [==============================] - 0s 12ms/step - loss: 5.7931 - acc: 0.0180
Epoch 8/10
8/8 [==============================] - 0s 12ms/step - loss: 5.7062 - acc: 0.0300
Epoch 9/10
8/8 [==============================] - 0s 12ms/step - loss: 5.6054 - acc: 0.0440
Epoch 10/10
8/8 [==============================] - 0s 12ms/step - loss: 5.5299 - acc: 0.0470
Epoch 1/10
8/8 [==============================] - 1s 12ms/step - loss: 5.45

### 다중 출력 모델

In [15]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(
    num_income_groups, activation='softmax', name='income'
)(x)
gender_prediction = layers.Dense(
    1, activation='sigmoid', name='gender'
)(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 posts (InputLayer)             [(None, None)]       0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, None, 256)    12800000    ['posts[0][0]']                  
                                                                                                  
 conv1d_9 (Conv1D)              (None, None, 128)    163968      ['embedding_4[0][0]']            
                                                                                                  
 max_pooling1d_4 (MaxPooling1D)  (None, None, 128)   0           ['conv1d_9[0][0]']               
                                                                                            

* 이런 모델을 훈련하려면 네트워크 출력마다 다른 손실 함수를 지정해야 한다.
* 경사하강법은 하나의 사칼라 값을 최소화하기 때문에 모델을 훈련하려면 이 손실들을 하나의 값으로 합쳐야 한다.
* 손실 값을 합치는 가장 간단한 방법은 모두 더하는 것이다.

In [16]:
model.compile(
    optimizer='rmsprop',
    loss=['mse', 'categorical_crossentropy', 'binary_crossentropy']
)

model.compile(
    optimizer='rmsprop',
    loss={
        'age': 'mse',
        'income': 'categorical_crossentropy',
        'gender': 'binary_crossentropy'
    }
)

* 손실 값이 많이 불균형하면 모델이 개별 손실이 가장 큰 작업에 치우쳐 표현을 최적화할 것이다.
* 그 결과 다른 작업들이 손해를 입는다.
* 손실 값이 최종 손실에 기여하는 수춘을 지정할 수 있다.

In [17]:
model.compile(
    optimizer='rmsprop',
    loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
    loss_weights=[0.25, 1., 10.]
)

model.compile(
    optimizer='rmsprop',
    loss={
        'age': 'mse',
        'income': 'categorical_crossentropy',
        'gender': 'binary_crossentropy'
    },
    loss_weights={
        'age': 0.25,
        'income': 1.,
        'gender': 10.
    }
)

In [18]:
import numpy as np
from keras.utils import to_categorical

num_samples = 10000
max_length = 1000

posts = np.random.randint(1, vocabulary_size, size=(num_samples, max_length))

age_targets = np.random.randint(0, 80, size=num_samples)
income_targets = np.random.randint(0, num_income_groups, size=num_samples)
gender_targets = np.random.randint(0, 2, size=num_samples)

income_targets = to_categorical(income_targets)

model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

model.fit(
    posts,
    {
        'age': age_targets,
        'income': income_targets,
        'gender': gender_targets
    },
    epochs=10,
    batch_size=64
)

Epoch 1/10
157/157 [==============================] - 4s 18ms/step - loss: 187.2975 - age_loss: 698.5448 - income_loss: 2.5952 - gender_loss: 1.0066
Epoch 2/10
157/157 [==============================] - 3s 16ms/step - loss: 146.0611 - age_loss: 541.4639 - income_loss: 2.3597 - gender_loss: 0.8335
Epoch 3/10
157/157 [==============================] - 3s 16ms/step - loss: 92.8698 - age_loss: 329.9888 - income_loss: 2.3602 - gender_loss: 0.8012
Epoch 4/10
157/157 [==============================] - 2s 16ms/step - loss: 58.1145 - age_loss: 191.7771 - income_loss: 2.3639 - gender_loss: 0.7806
Epoch 5/10
157/157 [==============================] - 2s 16ms/step - loss: 50.2089 - age_loss: 161.4221 - income_loss: 2.3652 - gender_loss: 0.7488
Epoch 6/10
157/157 [==============================] - 2s 16ms/step - loss: 42.3925 - age_loss: 129.3093 - income_loss: 2.3585 - gender_loss: 0.7707
Epoch 7/10
157/157 [==============================] - 2s 16ms/step - loss: 38.1918 - age_loss: 112.6464 - inco

### 층으로 구성된 비순환 유향 그래프
* 함수형 API를 사용하면 다중 입력이나 다중 출력 모델뿐만 아니라 내부 토폴로지가 복잡한 네트워크도 만들 수 있다.
* 케라스의 신경망은 층으로 구성된 어떤 **비순환 유향 그래프**(directed acyclic graph)도 만들 수 있다.

#### 인셉션 모듈
* **인셉션**(Inception)은 합성곱 신경망에서 인기 있는 네크워크 구조이다.
* **네트워크 안의 네트워크**(network-in-network) 구조에서 영감을 받아 2013~2014년에 크리스티안 세게디(Cristian Szegedy)와 그의 구글 동료들이 만들었다.
* 가장 기본적인 인셉션 모듈 형태는 3~4개의 가지를 가진다.
* 1x1 합성곱으로 시작해서 3x3 합성곱이 뒤따르고 마지막에 전체 출력 특성이 합쳐진다.
* 입력 x는 4D 텐서라고 가정.
```
from keras import layers
branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)
output = layers.concatenate(
    [branch_a, branch_b, branch_c, branch_d], axis=-1
)
```

#### 잔차 연결
* **잔차 연결**(residual connection)은 엑셉션을 포함하여 2015년 이후 등장한 네트워크 구조에 있는 그래프 형태의 네트워크 컴포넌트이다.
* 대규모 딥러닝 모델에서 흔히 나타나는 두 가지 문제인 그래디언트 소실과 표현 평목(representational bottleneck)을 해결했다.
* 일반적으로 10개 층 이상을 가진 모델에 잔차 연결을 추가하면 도움이 된다.
* 하의 층의 출력이 상위 층의 하캬성화 출력에 연결되는 것이 아니고 더해진다.
* 따라서 두 출력의 크기가 동일해야 한다.
* 입력 x가 4D 텐서라고 가정.
```
x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x])
```
---
```
from keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y =layers.AmxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])
```

### 층 가중치 공유
* 함수형 API의 중요한 또 하나의 기능은 층 객체를 여러 번 재사용할 수 있다는 것이다.
* 층 객체를 두 번 호출하면 새로운 층 객체를 만들지 않고 각 호출에 동일한 가중치를 재사용한다.
* 하나의 LSTM 층으로 양쭉을 모두 처리하는 것
* 이 LSTM 층의 표현(가중치)은 두 입력에 대해 함께 학습.
* **샴 LSTM**(Siames LSTM) 모델 또는 **공유 LSTM**.
```
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)
```

### 층과 모델
* 함수형 API에서는 모델을 층처럼 사용할 수 있다.
```
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None, include_top=False)
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate(
    [left_features, right_features], axis=-1
)
```

### 정리
* 차례대로 층을 쌓는 것 이상이 빌요할 때는 Sequential API를 사용하지 않는다.
* 함수형 API를 사용하여 다중 입력, 다중 출력, 복잡한 네트워크 토폴라지를 갖는 케라스 모델을 만드는 방법
* 다른 네트워크 가지에서 같은 층이나 모델 객체를 여러 번 호출하여 가중치를 재사용하는 방법